In [43]:
import scipy.stats as stats
import numpy as np
import math as math
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
def simulate_gbm(r,q,sigma,T,num_divisons,spot):
    delta_t=T*1.0/num_divisons
    S=spot
    nu=(r-q-(sigma**2/2))
    for i in range(num_divisons):
        z=np.random.standard_normal(1)
        S=S*math.exp(nu*delta_t+sigma*math.sqrt(delta_t)*z)
    return S


def black_scholes (cp, s, k, t, v, rf, div):
        """ Price an option using the Black-Scholes model.
        s: initial stock price
        k: strike price
        t: expiration time
        v: volatility
        rf: risk-free rate
        div: dividend
        cp: +1/-1 for call/put
        """

        d1 = (math.log(s/k)+(rf-div+0.5*math.pow(v,2))*t)/(v*math.sqrt(t))
        d2 = d1 - v*math.sqrt(t)

        optprice = (cp*s*math.exp(-div*t)*stats.norm.cdf(cp*d1)) - (cp*k*math.exp(-rf*t)*stats.norm.cdf(cp*d2))
        return optprice

    
def calculate_price_ci_european_call(r,q,sigma,T,spot,K,n_simulation_paths,alpha):
    Y=[]
    price=0
    discouting_factor=math.exp(-r*T) 
    for i in range(n_simulation_paths):
        S_T=simulate_gbm(r,q,sigma,T,400,spot)
        Y.append(discouting_factor*max(S_T-K,0))

    
    price=np.mean(Y)
    sample_std=np.std(Y)

    lower_bound_clt= price-((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    upper_bound_clt= price+((sample_std)/math.sqrt(n_simulation_paths))*stats.norm.ppf(1.0-(alpha/2))
    
    return (price,lower_bound_clt,upper_bound_clt)


    
    


In [45]:
#calculating price of the call using monte-carlo simulation

#option parameters
r=0.05
q=0.01
T=1
sigma=0.30
spot=100
K=90
true_price=black_scholes(1,100,90,1,0.30,0.05,0.01)
n_simulation_paths=10000
alpha=0.05 # 95% CI
print("----CLT CI------")
(price,lb,ub)=calculate_price_ci_european_call(r,q,sigma,T,spot,K,n_simulation_paths,alpha)
print("Price: "+str(price)+" LB: "+str(lb)+" UB:"+str(ub)+" Width of CI: "+str(ub-lb))
print("True BS Price: "+str(true_price))




----CLT CI------
Price: 18.835320599215375 LB: 18.33867571114563 UB:19.33196548728512 Width of CI: 0.9932897761394912
True BS Price: 18.958601352808394


In [46]:
from collections import defaultdict
import scipy as sp
# Calulating CI using Bootstrap
N= 10000 #samples to generate empirical distribution
B=100 #bootstrap samples
S_T=[simulate_gbm(r,q,sigma,T,400,spot) for _ in range(N)]
discouting_factor=math.exp(-r*T)
price_Fe=discouting_factor*np.mean([max(s-K,0) for s in S_T])

cdf=np.cumsum(np.ones(N)/N)
S_T_sorted=sorted(S_T)
theta_b=[]
Z_b=[]
Ss=[]
for i in range(B):
    ## we have to sample from F*
    Ss=(sp.interpolate.interp1d(np.insert(cdf,0,0),[0.00000001]+S_T_sorted,'linear')(np.random.uniform(0,1,N)))
    price_b=discouting_factor*np.mean([max(s-K,0) for s in Ss])
    theta_b.append(price_b)
    Z_b.append((price_b-price_Fe)**2)

mse= np.mean(Z_b)
theta_b_sorted=sorted(theta_b)


print("---------Bootstrap CI-----")
bootstrap_lb_ci=(price_Fe-math.sqrt(mse)*stats.norm.ppf(1-(alpha/2)))

bootstrap_ub_ci=(price_Fe+math.sqrt(mse)*stats.norm.ppf(1-(alpha/2)))

print("LB:"+str(bootstrap_lb_ci)+" UB: "+str(bootstrap_ub_ci)+ " Width of CI:"+str(bootstrap_ub_ci-bootstrap_lb_ci))

---------Bootstrap CI-----
LB:18.546199715849284 UB: 19.48021253487888 Width of CI:0.9340128190295971
